# Reprojection of WorldCover, Global Islands, and buffered GSHHG Continental Shoreline masks to Sentinel-2 granules

This notebook uses 3 global 10m zarr files for the three masks, partially generated 
by rasterisation of shapre files. 
It also uses dummy masks of all possible granules to define the target grid subsets.
The input data is provided in an S3 bucket on Creodias.
Credentials are to be provided in an .env file.

The output is a combined mask with 8 possible values for all combinations of land and water. The output is the input of a buffering step (next notebook) to generate the Sen2Water masks.

In [ ]:
import rioxarray as rio
import xarray as xr
import numpy as np
import scipy
from pyproj import CRS, Transformer
from pyproj.enums import TransformDirection
import math
import os
import sys
import s3fs
from dotenv import load_dotenv
print("software imported")

In [ ]:
granule = "32UME"
worldcover_path = "s3://sen2water-dev/worldcoverlandmask.zarr"
globalislands_path = "s3://sen2water-dev/globalislandsinlandmask.zarr"
continentalshoreline_path = "s3://sen2water-dev/continentalshorelinemask.zarr"
zeromask_path = f"s3://sen2water-dev/dummy-masks/{granule}-zeros.tif"
combined_mask_path = f"combined_mask-{granule}.tif"

load_dotenv()

endpoint_url = os.getenv("endpoint_url")
key = os.getenv("key")
secret = os.getenv("secret")
storage_options = {"endpoint_url": endpoint_url, "key": key, "secret": secret}

fs = s3fs.S3FileSystem(endpoint_url=endpoint_url, key=key, secret=secret)
print(fs.endpoint_url)

### Determine bounding box of granule in geographic worldcover grid ...

    hr = target dummy mask in granule's UTM (metric) projection
    wc = WorldCover land mask global zarr in 10m in geographic projection
    gi = Global Islands land mask global zarr rastered into same grid as wc
    cs = Continental Shoreline land mask global zarr rastered into same grid as wc

In [ ]:
# Use target dummy mask as representative for the target UTM grid

#hr = rio.open_rasterio(zeromask_path)  ## for local file system
with fs.open(zeromask_path) as f:
    hr = rio.open_rasterio(f)

hr_width = len(hr.x)
hr_height = len(hr.y)
hr_size = hr_height * hr_width
hr_step = 60.0
hr_left = hr.x.data[0]
hr_top = hr.y.data[0]

hr.plot()
print("dummy target granule with UTM (metric) coordinates")

In [ ]:
# Use WorldCover as representative for the source grid

wc = xr.open_dataset(
    worldcover_path, engine='zarr', mask_and_scale=False, storage_options=storage_options
)

# create transformation WGS84 -> UTM
hr_crs = CRS.from_cf(hr.spatial_ref.attrs)
wc_crs = CRS.from_cf(wc.spatial_ref.attrs)
wgs_to_utm = Transformer.from_crs(wc_crs, hr_crs)

# map border of granule to geographic grid
half_a_pixel = hr_step / 2
# extract image border (top, bottom, left, right) for transformation
hr_y = hr.y.values
hr_x = hr.x.values
hr_y_border = np.stack((np.tile(hr_y[0] + half_a_pixel, (hr_width)),
                        np.tile(hr_y[-1] - half_a_pixel, (hr_width)),
                        hr_y,
                        hr_y))
hr_x_border = np.stack((hr_x,
                        hr_x,
                        np.tile(hr_x[0] - half_a_pixel, (hr_width)),
                        np.tile(hr_x[-1] + half_a_pixel, (hr_width))))
# move four pixels of border into image corners
hr_x_border[0,0] -= half_a_pixel
hr_x_border[0,-1] += half_a_pixel
hr_x_border[1,0] -= half_a_pixel
hr_x_border[1,-1] += half_a_pixel

# transform border from UTM to WGS84 geographic coordinates
hr_lat,hr_lon = wgs_to_utm.transform(hr_x_border.flatten(), hr_y_border.flatten(), direction=TransformDirection.INVERSE)

# determine bounding box in geographic grid
hr_lat_min = np.min(hr_lat)
hr_lat_max = np.max(hr_lat)
# turn lon by first lon as reference to determine minmax lon of border
hr_lon_ref = hr_lon[0]
hr_lon_min = (np.min((hr_lon - hr_lon_ref + 180.0) % 360.0 - 180.0) + hr_lon_ref + 180.0) % 360.0 - 180.0
hr_lon_max = (np.max((hr_lon - hr_lon_ref + 180.0) % 360.0 - 180.0) + hr_lon_ref + 180.0) % 360.0 - 180.0

# determine minmax pixel coordinates of worldcover subset
wc_j_min = int(math.floor((90.0 - hr_lat_max) / 180.0 * wc.y.shape[0]))
wc_j_max = int(math.ceil((90.0 - hr_lat_min) / 180.0 * wc.y.shape[0]))
wc_i_min = int(math.floor((hr_lon_min + 180.0) / 360.0 * wc.x.shape[0]))
wc_i_max = int(math.ceil((hr_lon_max + 180.0) / 360.0 * wc.x.shape[0]))
# subsampling factor for geographic grid's longitude depending on latitude
i_step = int(1.0 / math.cos((hr_lat_min + hr_lat_max) / 2 * math.pi / 180.0))

print(f"extent {wc_j_min=} {wc_i_min=} {wc_j_max-wc_j_min=} {wc_i_max-wc_i_min=}")
print(f"latitude subsampling factor for geographic grids: {i_step}")
if wc_i_min > wc_i_max: print('crossing antimeridian')

# create subset of worldcover land mask
if wc_i_min <= wc_i_max:
    wc_subset = wc.landmask[wc_j_min:wc_j_max, wc_i_min:wc_i_max:i_step]
else:
    wc_subset = xr.concat((wc.landmask[wc_j_min:wc_j_max, wc_i_min:], wc.landmask[wc_j_min:wc_j_max, :wc_i_max]), dim='x')
    wc_subset = wc_subset[:,::i_step]

del hr_y, hr_x, hr_y_border, hr_x_border, hr_lat, hr_lon
print(f"\n{wc_subset=}")
print("WorldCover subset covering granule determined")

In [ ]:
print("plotting takes longer than ...")
wc_subset.plot(figsize=(7,3))
print("WorldCover 10m land mask subset covering granule in geographic projection")

### Determine mapping from geographic to UTM grid ...

In [ ]:
# transform land mask coordinates from WGS84 to UTM
wc_lat = np.tile(wc_subset.y.values, (len(wc_subset.x), 1)).T
wc_lon = np.tile(wc_subset.x.values, (len(wc_subset.y), 1))
wc_x, wc_y = wgs_to_utm.transform(wc_lat, wc_lon, direction=TransformDirection.FORWARD)
# determine pixel coordinates in HR-OC UTM grid for landmask subset
# label pixels outside of HR-OC mask granule
wc_j = ((hr_top - wc_y + hr_step / 2) / hr_step).astype(np.int32)
wc_i = ((wc_x - hr_left + hr_step / 2) / hr_step).astype(np.int32)

# map the landmask to the UTM grid
# bin_indices is the flattened wc coordinates mapped to hr grid bin indices
# landmask_values is the corresponding flattened wc values, 0 or 1
is_inside_granule = ((wc_j >= 0) & (wc_j < hr_height) & (wc_i >= 0) & (wc_i < hr_width)).astype(bool)
inside_outside_bin_indices = wc_j * hr_width + wc_i
bin_indices = inside_outside_bin_indices[is_inside_granule]
all_bin_counts = np.bincount(bin_indices, minlength=hr_size)

da = xr.DataArray(is_inside_granule, dims=wc_subset.dims, coords=wc_subset.coords, attrs=wc_subset.attrs)
da.plot(figsize=(7,3))

del wc_lat, wc_lon, wc_j, wc_i, inside_outside_bin_indices, wc, da

print(f"{bin_indices=}")
print(f"{all_bin_counts=}")
print("area of wc to be binned into target grid")

In [ ]:
### Reproject WorldCover, Global Islands, and GSHHG Continental Shoreline land masks to UTM grid

In [ ]:
wc_values = wc_subset.data[is_inside_granule]
wc_bin_counts = np.bincount(bin_indices[wc_values!=0], minlength=hr_size)
wc_bin_mask = 2 * wc_bin_counts >= all_bin_counts  # majority
wc_landmask = wc_bin_mask.reshape(hr_height, hr_width).astype(np.uint8)

da = xr.DataArray(wc_landmask.reshape((1, *wc_landmask.shape)), dims=hr.dims, coords=hr.coords, attrs=hr.attrs)
da.plot()
del wc_values, wc_bin_counts, wc_bin_mask, da
print("WorldCover land mask projected to target granule")

In [ ]:
gi = xr.open_dataset(
    globalislands_path, engine='zarr', mask_and_scale=False, storage_options=storage_options
)
if wc_i_min <= wc_i_max:
    gi_subset = gi.inlandmask[wc_j_min:wc_j_max, wc_i_min:wc_i_max:i_step]
else:
    gi_subset = xr.concat((gi.inlandmask[wc_j_min:wc_j_max, wc_i_min:], gi.inlandmask[wc_j_min:wc_j_max, :wc_i_max]), dim='x')
    gi_subset = gi_subset[:,::i_step]
gi_values = gi_subset.data[is_inside_granule]
gi_bin_counts = np.bincount(bin_indices[gi_values!=0], minlength=hr_size)
gi_bin_mask = 2 * gi_bin_counts >= all_bin_counts
gi_landmask = gi_bin_mask.reshape(hr_height, hr_width).astype(np.uint8)

da = xr.DataArray(gi_landmask.reshape((1, *gi_landmask.shape)), dims=hr.dims, coords=hr.coords, attrs=hr.attrs)
da.plot()
del gi_values, gi_bin_counts, gi_bin_mask, gi_subset, gi
print("Global Islands land mask projected to target granule")

In [ ]:
cs = xr.open_dataset(
    continentalshoreline_path, engine='zarr', mask_and_scale=False, storage_options=storage_options
)
if wc_i_min <= wc_i_max:
    cs_subset = cs.continentalmask[wc_j_min:wc_j_max, wc_i_min:wc_i_max:i_step]
else:
    cs_subset = xr.concat((cs.continentalmask[wc_j_min:wc_j_max, wc_i_min:], cs.continentalmask[wc_j_min:wc_j_max, :wc_i_max]), dim='x')
    cs_subset = cs_subset[:,::i_step]
cs_values = cs_subset.data[is_inside_granule]
cs_bin_counts = np.bincount(bin_indices[cs_values!=0], minlength=hr_size)
cs_bin_mask = 2 * cs_bin_counts >= all_bin_counts
cs_landmask = cs_bin_mask.reshape(hr_height, hr_width).astype(np.uint8)

da = xr.DataArray(cs_landmask.reshape((1, *cs_landmask.shape)), dims=hr.dims, coords=hr.coords, attrs=hr.attrs)
da.plot()
del cs_values, cs_bin_counts, cs_bin_mask, cs_subset, cs, is_inside_granule, all_bin_counts, bin_indices
print("Continental Shoreline GSHHG land mask projected to target granule")

In [ ]:
mask = wc_landmask + 2 * gi_landmask + 4 * cs_landmask
combinedmask_da = xr.DataArray(mask.reshape((1, hr_height, hr_width)), coords=hr.coords, dims=hr.dims, attrs=hr.attrs)
combinedmask_da.rio.to_raster(combined_mask_path, compress='LZW', tiled=True)
print(f"combined land mask stored in {combined_mask_path}")
combinedmask_da.plot(figsize=(12,9))

* 0 = ocean
* 1 = land in worldcover, missed islands in less accurate global islands
* 2 = inland water near coast
* 4 = inland river water
* 6 = inland water 
* 3 = coastal land, not in buffered continental shoreline
* 5 = inaccurate match of global islands and continental shoreline, should be 7
* 7 = all land

### done